<a href="https://colab.research.google.com/github/harryypham/MyMLPractice/blob/main/nlp/RNN_LSTM_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
batch_size = 64
lr = 3e-4
num_epochs = 5

In [3]:
train_set = datasets.MNIST(root="data", train=True, download=True, transform=transforms.ToTensor())
test_set = datasets.MNIST(root="data", train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:01<00:00, 5074315.79it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 134471.65it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 1271491.95it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 6316488.32it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [10]:
class RNN(nn.Module):
  def __init__(self, input_size, sequence_length, hidden_size, num_layers, num_classes):
    super().__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

    out, _ = self.rnn(x, h0)
    out = out.reshape(out.shape[0], -1)
    out = self.fc(out)

    return out

class LSTM(nn.Module):
  def __init__(self, input_size, sequence_length, hidden_size, num_layers, num_classes):
    super().__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

    out, _ = self.lstm(x, (h0, c0))
    out = self.fc(out[:, -1, :])

    return out

In [8]:
def train(model, criterion, optimizer, data_loader, num_epochs, device):
  model.train()
  losses = []
  for epoch in range(1, num_epochs+1):
    print(f"Epoch {epoch}: ")
    pbar = tqdm(data_loader, leave=True)
    for batch_idx, (input, target) in enumerate(pbar):
      input = input.to(device).squeeze(1)
      target = target.to(device)

      output = model(input)

      loss = criterion(output, target)
      losses.append(loss.item())
      optimizer.zero_grad()
      loss.backward()

      optimizer.step()

      pbar.set_postfix({"Loss": round(sum(losses)/len(losses), 4)})

@torch.no_grad()
def check_accuracy(model, data_loader, device):
  correct = total = 0

  model.eval()
  for input, target in data_loader:
    input = input.to(device).squeeze(1)
    target = target.to(device)

    output = model(input)

    _, preds = output.max(1)
    correct += (preds == target).sum()
    total += target.size(0)

  print(f"Accuracy: {correct/total*100:.2f}")
  model.train()


In [11]:
model = LSTM(input_size, sequence_length, hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

train(model, criterion, optimizer, train_loader, num_epochs, device)

Epoch 1: 


100%|██████████| 938/938 [00:11<00:00, 78.51it/s, Loss=0.563]


Epoch 2: 


100%|██████████| 938/938 [00:11<00:00, 82.97it/s, Loss=0.348]


Epoch 3: 


100%|██████████| 938/938 [00:11<00:00, 83.92it/s, Loss=0.261]


Epoch 4: 


100%|██████████| 938/938 [00:11<00:00, 83.41it/s, Loss=0.212]


Epoch 5: 


100%|██████████| 938/938 [00:11<00:00, 84.20it/s, Loss=0.18]


In [12]:
check_accuracy(model, test_loader, device)

Accuracy: 98.36
